In [ ]:
import networkx as nx
import numpy as np 
import ipynb.fs
from ipynb.fs.full.CNI import CNR
import pandas as pd 
from ipynb.fs.full.make_graphs import create_graph, node_attribute

G = create_graph()
nodelist = G.nodes

In [ ]:
def agg_feat():
    #CNI 
    _, CNR_log = CNR() 
    
    #NodeAttribute 
    node_att = node_attribute()
    
    #PageRank 
    pr = nx.pagerank(G) 
    
    #DegreeCentrality
    dc = nx.degree_centrality(G)
    
    #EigenvectorCentrality 
    ec = nx.eigenvector_centrality(G)
    
    #BetweennessCentrality 
    bc = nx.betweenness_centrality(G)
    
    #ClosennessCentrality 
    cc = nx.closeness_centrality(G)
    
    #LocalClusteringCoefficient 
    clustcoeff = nx.clustering(G) 
    
    #AverageNeighborDegree
    avnd = nx.average_neighbor_degree(G)
    averageND = dict()
    for k, v in avnd.items():
        if v == 1.0:
            v += 0.05
        averageND[k] = np.log10(v)        
    
    #Aggregating all attribute names in a list
    #feat_list = [CNR_log, node_att, clustcoeff, averageND, pr, dc, bc, cc, ec]
    feat_list = [CNR_log, dc, clustcoeff, ec]
    
    return feat_list

In [ ]:
def build_fmatrix():
    feat_list = agg_feat()
    
    feature_dict = dict()
    for n in nodelist: 
        temp = list()
        for feat in feat_list:
            temp.append(feat[n]) 
        
        feature_dict[n] = temp

    df = pd.DataFrame(feature_dict, columns=list(nodelist))
    print(df.head(5))
    feat_matrix = df.corr(method='pearson')      # 'pearson','spearman', 'kendall', or 'callable'
    
    return feat_matrix
print(build_fmatrix())


In [ ]:
def build_subgraph():
    nodes_list = list(nodelist)
    from sklearn.cluster import AffinityPropagation
    
    feat_matrix = build_fmatrix()
    
    training_data = feat_matrix.to_numpy(np.random.seed(42))
  
    model = AffinityPropagation(damping=0.8)    #0.75
    model.fit(training_data)
    
    # assign each data point to a cluster
    result = model.predict(training_data)

    # get all of the unique clusters
    clusters = np.unique(result)
    
    #Aggregate nodes into communities 
    subgraph_corpus = []
    for cluster in clusters:
        temp = []
        temp.append(cluster)    #Appending the label as first entry
        index = np.where(result == cluster)     #Array of array
        index = index[0]                        #Slicing out the array
        
        for idx in index:
            idx = int(idx)
            temp.append(nodes_list[idx])    #Convert graphview to list to make is hashable
        subgraph_corpus.append(temp)
    
    sub_labels = [sub[0] for sub in subgraph_corpus]
   
    return subgraph_corpus, sub_labels